# Import Data and Modules

In [100]:
# Imports
import numpy as np
import pandas as pd
from pickle import dump

from sklearn.preprocessing import OneHotEncoder

# Import train-test split function
from sklearn.model_selection import train_test_split

# Import the RandomForestClassifier and DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Import metrics for evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

# Import Visualization Modules
import matplotlib.pyplot as plt
import plotly.express as px

In [101]:
df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv")
mini = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test_mini.csv")

# Data Exploration

### Exploring unknown values

In the categorical columns there are missing values labeled as 'unknown'. Here we find them so we can figure out what to do with them.

In [102]:
unknown_mask = df.isin(['unknown'])

# Filter the DataFrame to show rows with 'unknown' values
unknown_rows = df[unknown_mask.any(axis=1)]
unknown_rows.shape

(9592, 20)

Going a little deeper, here are the counts of unkown values in each categorical column. We decided that we will leave these values in since removing them did not substantially improve model performance.

In [103]:
# Define a custom function to count "unknown" values
def count_unknown(series):
    return (series == 'unknown').sum()

# Apply the function to each column using agg
unknown_counts = df.agg(count_unknown).sort_values(ascending=False).reset_index()
unknown_counts = unknown_counts[unknown_counts[0]> 0]
unknown_counts

,index,0
0,default,7725
1,education,1535
2,housing,894
3,loan,894
4,job,294
5,marital,69


### Making connections

Here we show some percentages to try and make connections between who has a term deposit and what key demographics they have

#### Percentage of people who have term deposits, by job

In [104]:
((df[df['y'] == 'yes']['job'].value_counts() / df['job'].value_counts()) * 100).sort_values(ascending=False)

job
student          31.399748
retired          25.228162
unemployed       14.676889
admin.           13.127660
unknown          11.904762
management       11.442598
technician       10.863831
self-employed    10.753532
housemaid         9.979210
entrepreneur      8.639144
services          8.258174
blue-collar       6.819546
Name: count, dtype: float64

#### Percentage of people who have term deposits, by age


In [105]:
((df[df['y'] == 'yes']['age'].value_counts() / df['age'].value_counts()) * 100).sort_values(ascending=False)

age
98    100.000000
89    100.000000
87    100.000000
92     75.000000
86     71.428571
         ...    
49      6.710526
47      6.521739
91           NaN
94           NaN
95           NaN
Name: count, Length: 78, dtype: float64

# Data Cleaning and Encoding

### Map y values to 0 and 1

In [106]:
df['y'] = df['y'].map({"no": 0, "yes": 1})
df['y'].value_counts()

y
0    32861
1     4208
Name: count, dtype: int64

### Drop duplicates

In [107]:
print('Before', len(df))
df = df.drop_duplicates()
print('After', len(df))

Before 37069
After 35582


### Reduce distinct values for Eurob3r

Eurob3r has too many distinct values. Reducing the amount of these through rounding will help us to lessen the number of columns we have to deal with when we one-hot encode our data.

In [108]:
# print('Rounding euribor3m')
# print('Distinct vaues before:', len(df['euribor3m'].value_counts()))
# df['euribor3m'] = df['euribor3m'].round(1)
# print('Distinct values after:', len(df['euribor3m'].value_counts()))

### Create X and Y datasets

In [109]:
X = df.drop("y", axis=1)
y = df["y"]

### One-hot encode categorical columns

In [110]:
X = pd.get_dummies(X).drop("default_yes", axis=1)

# Creating the Model

### Tune the model parameters

Here we have code to programatically evaluate model performance using different parameters. It is commented out because it takes a long time to run.

In [111]:
# full = []
# i = 0
# for test in np.arange(.1, .4, .05):
#     for samp_size in range(50, 500, 100):
#         for md in range(10, 200, 25):
#             for wght in np.arange(.0, .55, .05):
#                 acc, recall, pre = get_nums(test, md, samp_size, wght)
#                 nummies = [test, samp_size, md, wght, acc, recall, pre]
#                 full.append(nummies)
#                 print(i)
#                 i +=1

# big = pd.DataFrame(full, columns=["test", "samp_size", "md", "wght", "acc", "recall", "pre"])
# big.to_csv("nummies.csv")


In [112]:
# full = []
# for test in np.arange(.1, .4, .05):
#     for samp_size in range(50, 500, 100):
#         for md in range(10, 200, 25):
#             for wght in np.arange(.0, .55, .05):
#                 nummies = [test, samp_size, md, wght]
#                 full.append(nummies)

# pd.DataFrame(full, columns=["test", "samp_size", "md", "wght"])

In [113]:
# big[(big["recall"] > .7) & (big["acc"] > .8)]

### Train the model

In [116]:
# One Hot encoding, data splitting, model training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=42)

# parameters
md = 85
min_samp = 150
class_weight = np.where(y_train == 0, 0.15, 1.0)

t_model = DecisionTreeClassifier(random_state=42, max_depth=md, min_samples_split=min_samp)

t_model.fit(X_train, y_train, sample_weight=class_weight)

tree_predict = t_model.predict(X_test)

tree_acc = accuracy_score(y_test, tree_predict)
tree_recall = recall_score(y_test, tree_predict, pos_label=1)
tree_pre = precision_score(y_test, tree_predict, pos_label=1)


print(f'Accuracy: {tree_acc}')
print(f'Recall: {tree_recall}')
print(f'Precision: {tree_pre}')

Accuracy: 0.802547770700637
Recall: 0.7016129032258065
Precision: 0.33358895705521474


# Predict Mini Holdout

In [119]:
# Do same transformations as on the training set
mini_onehot = pd.get_dummies(mini)

predictions = t_model.predict(mini_onehot)

# Convert the predictions to a dataframe and label the column 'y'
predictions_df = pd.DataFrame(predictions, columns = ['y'])

# Replace PUTTEAMNUMBERHERE with your team
predictions_df.to_csv("team2-module2-predictions.csv", index=False)

print(predictions_df.value_counts())

y
0    306
1    104
Name: count, dtype: int64


# Pickling the model

In [ ]:
with open("Decision Tree.pkl", "wb") as f:
    dump(t_model, f, protocol=5)